In [ ]:
import json

with open('weather-gov.json', 'r') as file:
    data = json.load(file)



In [ ]:
print(data['features'][0]['properties']['sent'])
print(data['features'][0]['properties']['effective'])
print(data['features'][0]['properties']['onset'])
print(data['features'][0]['properties']['expires'])
print(data['features'][0]['properties']['messageType'])
print(data['features'][0]['properties']['severity'])
print(data['features'][0]['properties']['urgency'])
print(data['features'][0]['properties']['event'])
print(data['features'][0]['properties']['sender'])
print(data['features'][0]['properties']['senderName'])
print(data['features'][0]['properties']['headline'])
print(data['features'][0]['properties']['description'])
print(data['features'][0]['properties']['instruction'])
print(data['features'][0]['properties']['response'])

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType
import requests

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("WeatherAlerts") \
    .getOrCreate()

# Function to fetch weather alerts using Weather.gov API
def fetch_weather_alerts(lat, lon):
    try:
        url = f"https://api.weather.gov/alerts/active?point={lat},{lon}"
        headers = {"User-Agent": "MyWeatherApp (your-email@example.com)"}
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            alerts = data.get('features', [])
            return "; ".join([alert['properties']['headline'] for alert in alerts])
        else:
            return f"Error: {response.status_code}"
    except Exception as e:
        return str(e)

# Register the function as a UDF
fetch_weather_alerts_udf = udf(fetch_weather_alerts, StringType())

# Example DataFrame with latitude and longitude
data = [
    {"latitude": 25.7617, "longitude": -80.1918},  # Miami, FL
    {"latitude": 34.0522, "longitude": -118.2437},  # Los Angeles, CA
    {"latitude": 31.6302, "longitude": -91.5551},  # Ferriday, LA
]
df = spark.createDataFrame(data)

# Add a column with weather alerts
df_with_alerts = df.withColumn("alerts", fetch_weather_alerts_udf(col("latitude"), col("longitude")))

# Show results
df_with_alerts.show(truncate=False)

# Stop SparkSession
spark.stop()
